In [91]:
#import words.json
import json
import numpy as np
from string import ascii_lowercase as alphabet


def split(word):
    return [char for char in word]

knownLetters=split("gho")#all yellow or green
usedLetters=np.array(split("ghost")) #all used letters
usedLettersWithUnknownPosition= [[],[],[],[],[]] #only yellow used letters in correct position
lettersWithKnownPosition=["g","h","o","",""]#green

In [92]:
print(knownLetters)
print(usedLetters)
print(usedLettersWithUnknownPosition)
print(lettersWithKnownPosition)


['g', 'h', 'o']
['g' 'h' 'o' 's' 't']
[[], [], [], [], []]
['g', 'h', 'o', '', '']


In [93]:

with open('Words.json') as f:
    allWordleWords = json.load(f)


def filterWordsContainingAllKnownLetters(allWordleWords,knownLetters):
    filteredWordleWords=[]
    for word in allWordleWords:
        if all(letter in word for letter in knownLetters):
            filteredWordleWords.append(word)
    return filteredWordleWords

def filterWordsAccordingToKnownPositions(remainingWordleWords,lettersWithKnownPosition):
    if lettersWithKnownPosition == ["","","","",""]:
        return remainingWordleWords
    filteredWordleWords=[]
    #append all words that contain all letters in correct position
    for word in remainingWordleWords:
        allLettersInWordFit=[False]*5
        for letterIndex, letter in enumerate(word):
            if lettersWithKnownPosition[letterIndex] == "":
                allLettersInWordFit[letterIndex]=True
            elif letter == lettersWithKnownPosition[letterIndex]:
                allLettersInWordFit[letterIndex]=True
        if all(allLettersInWordFit):
            filteredWordleWords.append(word)
    return filteredWordleWords


    # for word in remainingWordleWords:
    #     allConditionsMet=False
    #     for index, letter in enumerate(word):
    #         if lettersWithKnownPosition[index]=="":
    #             pass
    #         elif letter in lettersWithKnownPosition[index]:
    #             allConditionsMet=True
    #         else:
    #             allConditionsMet=False
    #     if allConditionsMet:
    #         filteredWordleWords.append(word)
    return filteredWordleWords

#determine unique letters that cannot be in word
def getAvailableLetters(knownLetters,usedLetters):
    availableLetters=[]
    for letter in alphabet:
        if letter in knownLetters or letter not in usedLetters:
            availableLetters.append(letter)
    return availableLetters

def filterWordsAccordingToAvailableLetters(remainingWordleWords,availableLetters):
    filteredWordleWords=[]
    for word in remainingWordleWords:
        if all(letter in availableLetters for letter in word):
            filteredWordleWords.append(word)
    return filteredWordleWords

def availableLettersAccordingToPosition(usedLettersWithUnknownPosition):
    availableLetters=[[],[],[],[],[]]
    for letter in alphabet:
        for index, usedLetterArray in enumerate(usedLettersWithUnknownPosition):
            if letter not in usedLetterArray:
                availableLetters[index].append(letter)
    return availableLetters

def filterWordsAccordingToKnownLetterPositions(remainingWordleWords,availableLettersWithPosition):
    filteredWordleWords=[]
    
    for word in remainingWordleWords:
        allLettersAvailable=True
        for index, letter in enumerate(word):
            if letter not in availableLettersWithPosition[index]:
                allLettersAvailable=False
        if allLettersAvailable:
            filteredWordleWords.append(word)
    return filteredWordleWords

def split(word):
    return [char for char in word]


In [94]:
availableLettersWithPosition = availableLettersAccordingToPosition(usedLettersWithUnknownPosition)
availableLetters = getAvailableLetters(knownLetters,usedLetters)
remainingWordleWords = filterWordsContainingAllKnownLetters(allWordleWords,knownLetters)
remainingWordleWords = filterWordsAccordingToKnownPositions(remainingWordleWords,lettersWithKnownPosition)
remainingWordleWords = filterWordsAccordingToAvailableLetters(remainingWordleWords,availableLetters)
remainingWordleWords = filterWordsAccordingToKnownLetterPositions(remainingWordleWords,availableLettersWithPosition)

['g', 'h', 'o', '', '']


In [95]:
letterCountDict = {}
for letter in alphabet:
    letterCountDict[letter] = [0,0,0,0,0]

In [96]:
for word in remainingWordleWords:
    for index, letter in enumerate(word):
        letterCountDict[letter][index] += 1

In [97]:
normalizedLetterCountDict = {}
for letter in alphabet:
    normalizedLetterCountDict[letter] = [0,0,0,0,0]
    for index, count in enumerate(letterCountDict[letter]):
        normalizedLetterCountDict[letter][index] = count/len(remainingWordleWords)


In [98]:
#determine probability duplicate letters in remaining word list index is letter number of occurrences
letterMultiplesArray = {}
for letter in alphabet:
    letterMultiplesArray[letter] = [0,0,0,0,0]

In [99]:
for word in remainingWordleWords:
    for letter in word:
        letterMultiplesArray[letter][word.count(letter)-1] += 1

In [100]:
# normalise letterMultiplesArray
normalizedLetterMultiplesDict = {}
for letter in alphabet:
    normalizedLetterMultiplesDict[letter] = [0,0,0,0,0]
    for index, count in enumerate(letterMultiplesArray[letter]):
        normalizedLetterMultiplesDict[letter][index] = count/len(remainingWordleWords)

In [101]:
#rate all words in remainingWordleWords return dictionary with word as key and score as value
wordScoreDict = {}
for word in remainingWordleWords:
    score = 0
    for index, letter in enumerate(word):
        score += normalizedLetterCountDict[letter][index]
    
    wordScoreDict[word] = score

In [102]:
#rate all words according to multiple of letter
wordScoreMultipleDict = {}
for word in remainingWordleWords:
    score = 0
    for index, letter in enumerate(word):
        score += normalizedLetterMultiplesDict[letter][index]
    
    wordScoreMultipleDict[word] = score

In [103]:
#sort wordScoreDict by value
sortedWordScoreDict = sorted(wordScoreDict.items(), key=lambda x: x[1], reverse=True)
print(sortedWordScoreDict)

[('hough', 2.5), ('bough', 2.2777777777777777), ('cough', 2.2777777777777777), ('dough', 2.2222222222222223), ('rough', 2.2222222222222223), ('lough', 2.2222222222222223), ('gouch', 1.9444444444444444), ('hongi', 1.777777777777778), ('hoagy', 1.7222222222222223), ('gonch', 1.6666666666666667), ('hoing', 1.5000000000000002), ('hogan', 1.4444444444444446), ('hogen', 1.388888888888889), ('brogh', 1.2777777777777777), ('cohog', 1.2222222222222223), ('ohing', 0.611111111111111), ('ghoul', 0.5), ('ogham', 0.4444444444444444)]


In [104]:
sortedWordScoreMultipleDict = sorted(wordScoreMultipleDict.items(), key=lambda x: x[1], reverse=True)
print(sortedWordScoreMultipleDict)

[('ghoul', 1.1111111111111112), ('gonch', 1.1111111111111112), ('gouch', 1.1111111111111112), ('hoagy', 1.0555555555555556), ('hogan', 1.0555555555555556), ('hogen', 1.0555555555555556), ('hoing', 1.0555555555555556), ('hongi', 1.0555555555555556), ('hough', 1.0555555555555556), ('ohing', 1.0555555555555556), ('ogham', 0.9444444444444444), ('cough', 0.3333333333333333), ('cohog', 0.3333333333333333), ('bough', 0.2222222222222222), ('rough', 0.2222222222222222), ('lough', 0.2222222222222222), ('dough', 0.16666666666666666), ('brogh', 0.1111111111111111)]


In [105]:
totalScoreDict = {}
for word in remainingWordleWords:
    for index, letter in enumerate(word):
        #count letter in word
        multiple=word.count(letter)
        totalScoreDict[word] = normalizedLetterCountDict[letter][index] + normalizedLetterMultiplesDict[letter][multiple]

In [106]:
#sort totalScoreDict by value
sortedTotalScoreDict = sorted(totalScoreDict.items(), key=lambda x: x[1], reverse=True)
print(sortedTotalScoreDict)

[('bough', 0.6111111111111112), ('cough', 0.6111111111111112), ('dough', 0.6111111111111112), ('rough', 0.6111111111111112), ('brogh', 0.6111111111111112), ('gonch', 0.6111111111111112), ('gouch', 0.6111111111111112), ('lough', 0.6111111111111112), ('hough', 0.5), ('cohog', 0.16666666666666666), ('hoing', 0.16666666666666666), ('ohing', 0.16666666666666666), ('hogan', 0.1111111111111111), ('hogen', 0.1111111111111111), ('ghoul', 0.05555555555555555), ('hoagy', 0.05555555555555555), ('hongi', 0.05555555555555555), ('ogham', 0.05555555555555555)]


In [107]:
# #remainingWordleWords = sortRemainingWordleWordsAccordingToLetterFrequencyAndPosition(remainingWordleWords)
# def calculateLetterScoreDict(remainingWordleWords):
#     letters={}
#     for word in remainingWordleWords:
#         letterLevel = getLetterLevel(word)
#         for index, letter in enumerate(word):
#             #if letter in letters and letter not in duplicateLetters:
#             if letter in letters:
#                 letters[letter] = letterFrequencyArray(letter,letterLevel)
#             else:
#                 letters[letter] = 1
#     # for letter in letters:
#     #     letters[letter] = letters[letter]/len(remainingWordleWords)
#     return letters

# def letterFrequencyArray(letter,letterLevel):
#     "returns the frequency of letter in word in an array of five elements"
#     letterCount = letterLevel[letter]
#     for position in range(letterCount):
#         letterFrequencyArray[position] += 1
#     return letterFrequencyArray

# def getLetterLevel(word):
#     "returns a dictionary with count of letters in word as values and letter as key"
#     countedRepeatedLetters={}
#     for letter in word:
#         if letter in countedRepeatedLetters:
#             countedRepeatedLetters[letter] += 1
#         else:
#             countedRepeatedLetters[letter] = 1
#     return countedRepeatedLetters



# #give score to words according to letter scores
# def scoreWords(remainingWordleWords,letterScores):
#     wordScores={}
#     for word in remainingWordleWords:
#         wordScores[word] = sum(letterScores[letter] for letter in word)
#     return wordScores

# letterScores = calculateLetterScoreDict(remainingWordleWords)